<a href="https://colab.research.google.com/github/Namtk214/OCR_project/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.1/107.7 GB disk)


In [2]:
import os
import numpy as np
import timm
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image

from ultralytics import YOLO


In [4]:
yolo = YOLO("/content/best.pt")

In [5]:
# Model
class CRNN(nn.Module):
    def __init__ (self, vocav_size, hidden_size, n_layers, dropout=0.2, unfreeze_layers=3):
        super(CRNN, self).__init__()

        backbone = timm.create_model(
            "resnet152",
            in_chans=1,
            pretrained=True
        )
        modules = list(backbone.children())[:-2]
        modules.append(nn.AdaptiveAvgPool2d((1, None)))
        self.backbone = nn.Sequential(*modules)

        for parameter in self.backbone[-unfreeze_layers:].parameters():
            parameter.requires_grad = False

        self.mapSeq = nn.Sequential(
            nn.Linear(2048, 512), nn.ReLU(), nn.Dropout(dropout))

        self.GRU = nn.GRU(
            input_size=512,
            hidden_size=hidden_size,
            num_layers=n_layers,
            dropout=dropout if n_layers >1 else 0,
            bidirectional=True,
            batch_first=True
        )

        self.layer_norm = nn.LayerNorm(hidden_size*2)
        self.out = nn.Sequential(nn.Linear(hidden_size*2, vocab_size), nn.LogSoftmax(dim=2))

    @torch.autocast(device_type="cuda")
    def forward(self, x):
        x = self.backbone(x)
        x = x.permute(0, 3, 1, 2)
        x = x.view(x.size(0), x.size(1), -1)
        x = self.mapSeq(x)
        x, _ = self.GRU(x)
        x = self.layer_norm(x)
        x = self.out(x)
        x = x.permute(1, 0, 2)
        return x



/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [6]:
chars = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
vocab_size = len(chars)
char_to_idx = {char: idx + 1 for idx, char in enumerate(sorted(chars))}
idx_to_char = {idx: char for idx, char in enumerate(chars)}

hidden_size = 256
n_layers = 2
dropout = 0.2
unfreeze_layers = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CRNN(vocab_size, hidden_size, n_layers, dropout, unfreeze_layers).to(device)
model.load_state_dict(torch.load("/content/ocr_crnn.pt"))



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory